In [ ]:
import pandas as pd

Load Data Files

In [ ]:
def load_data(file_path: str) -> pd.DataFrame:
    """Load data from a CSV file."""
    return pd.read_csv(file_path)

In [ ]:
# Anime dataset
anime_filtered = load_data('../data/clean/anime-dataset-filtered.csv')

# Recommendation datasets
user_based = load_data('../data/prediction/users-recommendations.csv')
content_based = load_data('../data/prediction/content-recommendations.csv')

# Validation dataset for 2025
watch_shrunk_25 = load_data('../data/clean/users-score-shrunk-2025.csv')

Transform the Anime Dataset

In [ ]:
def transform_anime_data(anime_df: pd.DataFrame) -> pd.DataFrame:
    """Filter relevant columns from anime data."""
    anime_df['Categories'] = (anime_df[['Genres', 'Explicit Genres', 'Themes', 'Demographics']].apply(lambda row: ', '.join(
            dict.fromkeys(sum([[s.strip() for s in x.split(',')] for x in row if pd.notna(x)], []))),
            axis=1)
    )
    return anime_df.set_index('anime_id')

Check for Missing Users

In [ ]:
missing_users_user_based = set(user_based['user_id']).difference(set(watch_shrunk_25['user_id']))
missing_users_content_based = set(content_based['user_id']).difference(set(watch_shrunk_25['user_id']))
print(f"Missing users in user-based recommendations: {missing_users_user_based}")
print(f"Missing users in content-based recommendations: {missing_users_content_based}")